# Read the News Table from the Lakehouse DB


In [2]:
df = spark.sql("SELECT * FROM Bingnews_LH.tbl_latest_news")
display(df)

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 51517be2-5d07-4c39-ba7c-9bbd64fb31df)

# Import Synapse ML

In [3]:
pip install synapseml

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 5, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.7/584.7 kB 2.8 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


# Import Synapse ML

In [4]:
import synapse.ml.core
from synapse.ml.cognitive.language import AnalyzeText

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 6, Finished, Available, Finished)

/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/synapse/ml/internal_utils/import_hooks/__init__.py:303: DeprecationWarning: Importing 'AddDocuments' from 'synapse.ml.cognitive' is deprecated. Use 'synapse.ml.services' instead.
  module = self.loader.load_module(fullname)
/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/synapse/ml/internal_utils/import_hooks/__init__.py:303: DeprecationWarning: Importing 'AddressGeocoder' from 'synapse.ml.cognitive' is deprecated. Use 'synapse.ml.services' instead.
  module = self.loader.load_module(fullname)
/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/synapse/ml/internal_utils/import_hooks/__init__.py:303: DeprecationWarning: Importing 'AnalyzeBusinessCards' from 'synapse.ml.cognitive' is deprecated. Use 'synapse.ml.services' instead.
  module = self.loader.load_module(fullname)
/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packag

# Configure Synapse ML

In [5]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description") #on which column we need to perform sentiment analysis
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 7, Finished, Available, Finished)

# Apply the model

In [6]:
#Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 8, Finished, Available, Finished)

In [7]:
display(result)

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 990acc1c-915a-4c17-a70b-1ad667de53f0)

# Grab sentiment from the response

In [8]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 10, Finished, Available, Finished)

In [9]:
display(sentiment_df)

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 37131f78-a95c-4d6d-87e6-90ecb2d7130e)

# Remove unwanted columns

In [10]:
#dropping the error and response columns
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 12, Finished, Available, Finished)

In [11]:
display(sentiment_df_final)

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9c6994d1-0dfe-46a2-a02c-1e6762e1394b)

In [12]:
sentiment_df_final.printSchema()

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 14, Finished, Available, Finished)

root
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image: string (nullable = true)
 |-- provider: string (nullable = true)
 |-- datePublished: string (nullable = true)
 |-- sentiment: string (nullable = true)



# Changin the date column datatype

In [13]:
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MM-yyyy"))

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 15, Finished, Available, Finished)

# Type 1 Merge

In [15]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'Bingnews_LH.tbl_sentiment_analysis'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view

                    ON source_view.url = target_table.url
                    
                    WHEN MATCHED AND 
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished   

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, c65fce8d-de90-4a15-bd8b-d57c6608a7ab, 17, Finished, Available, Finished)

In [54]:
%%sql

SELECT count(*) from Bingnews_LH.tbl_sentiment_analysis

StatementMeta(, e2138975-db5d-43a2-b373-5706d689a79e, 56, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [55]:
display(sentiment_df_final)

StatementMeta(, e2138975-db5d-43a2-b373-5706d689a79e, 57, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 42732783-c364-4fd0-8c0c-45692a05a109)